# สำรวจชุดข้อมูลขายของออนไลน์

**จุดประสงค์การเรียนรู้**
1. การอ่านไฟล์ Excel และการนำข้อมูลตารางมาต่อกัน
2. การตรวจสอบความไม่สอดคล้องของข้อมูล
3. การตรวจสอบและจัดการข้อมูลที่ขาดหาย (Missing values)
4. คำนวณภาพรวมของข้อมูลเป็นกลุ่ม (`groupby`)
5. จัดการข้อมูลเกี่ยวกับเวลา
6. เรียนรู้การวาดกราฟเพิ่มเติมจากคาบที่ผ่านมา


## เริ่มต้นบทเรียน

ร้านขายของออนไลน์แห่งหนึ่งได้บันทึกรายการธุรกรรม (transaction) ระหว่างวันที่ 1 กรกฎาคม 2011 ถึง 31 สิงหาคม 2011 ร้านค้ารายนี้จดทะเบียนในสหราชอาณาจักร ขายของใช้เก๋ ๆ ไม่ซ้ำใคร และถึงแม้จะไม่มีหน้าร้าน แต่กลุ่มลูกค้าก็เข้าถึงจากหลากหลายประเทศ

รายละเอียดของแต่ละรายการธุรกรรม มีดังนี้

| คอลัมน์ | คำอธิบาย |
| :-- | :-- |
| `InvoiceNo` | เลขธุรกรรม (ตัวเลข 6 หลัก) โดยหากมีการขึ้นต้นด้วย `C` จะหมายถึงการยกเลิกสินค้า |
| `StockCode` | รหัสสินค้า (ตัวเลข 5 หลัก) |
| `Description` | ชื่อสินค้า |
| `Quantity` | จำนวนชิ้นที่ซื้อ  |
| `InvoiceDate` | วันและเวลาของธุรกรรม  |
| `UnitPrice` | ราคาต่อหน่วย |
| `CustomerID` | รหัสลูกค้า (ตัวเลข 5 หลัก) |
| `Country` | ประเทศที่อยู่ของลูกค้า |




## ดาวน์โหลดข้อมูล

ในขั้นแรก จะดาวน์โหลดข้อมูลและบันทึกไว้ในไฟล์ชื่อ `data.xlsx`

In [ ]:
# ใช้คำสั่ง wget ซึ่งเป็นคำสั่ง linux (ไม่ใช่โค้ดภาษา Python) ใน
!wget -c https://github.com/kasemsit/269382/raw/main/dataset/online_retail_2months.xlsx -O data.xlsx

--2023-11-13 05:00:55--  https://github.com/kasemsit/269382/raw/main/dataset/online_retail_2months.xlsx
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kasemsit/269382/main/dataset/online_retail_2months.xlsx [following]
--2023-11-13 05:00:56--  https://raw.githubusercontent.com/kasemsit/269382/main/dataset/online_retail_2months.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



**คำอธิบาย:** โค้ด `!wget` ข้างต้นไม่ใช่ภาษา Python แต่เป็นการเรียกใช้โปรแกรม `wget` ที่เป็นโปรแกรมของระบบปฏิบัติการ Linux ผ่านทาง command line ดังสังเกตุได้จากเครื่องหมาย `!` นำหน้า ที่เป็นการบอกให้ Google Colab เรียกโปรแกรม `wget` โดยสิ่งที่ตามมา มีความหมายคือ
- `-c` หมายถึงให้ดาวน์โหลดแบบ resume (คือไม่ต้องดาวน์โหลดใหม่ถ้ามีไฟล์อยู่แล้ว)
- `https://github.com/kasemsit/269382/raw/main/dataset/online_retail_2months.xlsx` คือ URL ของไฟล์
- `-O data.xlsx` หมายถึง ให้ดาวน์โหลดแล้วบันทึกไฟล์ในชื่อ `data.xlsx`

นอกจากนี้ นักศึกษาสามารถเรียกใช้โปรแกรมอื่น ๆ นอกจาก `wget` ได้เช่น `!ls` เพื่อดูว่าโฟลเดอร์ปัจจุบันมีไฟล์อะไรบ้าง หรือแม้กระทั่ง `!unzip ชื่อไฟล์` เพื่อทำการแตกไฟล์ zip เป็นต้น

**ลองทำ:**
1. ดาวน์โหลดไฟล์ Excel ไปที่คอมพิวเตอร์ของนักศึกษาด้วย และอ่านไฟล์ด้วยโปรแกรม Excel
2. ไฟล์ Excel ที่เก็บข้อมูล มีกี่ Sheet และชื่อ Sheet อะไรบ้าง

ในขั้นตอนต่อจากนี้ จะเป็นการอ่านไฟล์ Excel ใน Google Colab ด้วย `Pandas`

## อ่านข้อมูลจากไฟล์ Excel

เมื่อดาวน์โหลดข้อมูลสำเร็จและบันทึกไว้ในไฟล์ `data.xlsx` จากนั้นจึงอ่านไฟล์เข้ามาด้วยคำสั่ง `read_excel()` ของ `Pandas` [(ดูคู่มือ)](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) ซึ่งมีรูปแบบ ดังนี้

```python
pd.read_excel(ชื่อไฟล์, sheet_name=..., header=..., skiprows=...)
```

โดยที่พารามิเตอร์ที่สำคัญที่ใช้บ่อย ได้แก่

| พารามิเตอร์  | การกำหนดค่า | ค่า default |
|:---|:---|:--|
|`sheet_name`| ชื่อ sheet ที่ต้องการอ่าน หรือตัวเลขลำดับที่ของ sheet |  `0` (sheet แรก) |
| `skiprows` | เลขแถวที่จะให้เริ่มอ่านเป็นแถวแรก (แถวแรกนับเป็นแถวที่ `0`) | `0` (แถวแรกของ sheet) |
| `header` | ให้ระบุ `header=None` หากไม่มีชื่อคอลัมน์ใน sheet  | `0` (แถวแรกของ sheet เป็น `header`) |
| `nrows` | จำนวนแถวที่ต้องการอ่าน (นับจาก `header`) | `None` (อ่านทุกแถวจนจบ sheet) |


**อ่าน Sheet เดือนกรกฏาคม**

In [ ]:
# บรรทัดนี้ไม่ใช่คำสั่ง Python แต่เป็นคำสั่งพิเศษ (magic command) ของ Google Colab (หรือ iPython) ที่ใช้เคลียร์ตัวแปรในหน่วยความจำ
%reset -f

In [ ]:
import pandas as pd
data_jul = pd.read_excel('data.xlsx',
                      sheet_name='July',  # เดือนกรกฎาคม
                      skiprows=3,
                      )

In [ ]:
data_jul.head(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,558638,84836,ZINC METAL HEART DECORATION,12,2011-07-01 08:16:00,1.25,16317.0,United Kingdom
1,558638,71459,HANGING JAM JAR T-LIGHT HOLDER,24,2011-07-01 08:16:00,0.85,16317.0,United Kingdom
2,558638,22784,LANTERN CREAM GAZEBO,3,2011-07-01 08:16:00,4.95,16317.0,United Kingdom
3,558638,23145,ZINC T-LIGHT HOLDER STAR LARGE,12,2011-07-01 08:16:00,0.95,16317.0,United Kingdom


**อ่าน Sheet เดือนสิงหาคม**

In [ ]:
import pandas as pd
data_aug = pd.read_excel('data.xlsx',
                      sheet_name='August',  # เดือนสิงหาคม
                      skiprows=3,
                      )

In [ ]:
data_aug.head(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,561904,22075,6 RIBBONS ELEGANT CHRISTMAS,96,2011-08-01 08:30:00,1.45,17941.0,United Kingdom
1,561904,85049E,SCANDINAVIAN REDS RIBBONS,156,2011-08-01 08:30:00,1.06,17941.0,United Kingdom
2,561905,21385,IVORY HANGING DECORATION HEART,24,2011-08-01 09:31:00,0.85,14947.0,United Kingdom
3,561905,84970L,SINGLE HEART ZINC T-LIGHT HOLDER,12,2011-08-01 09:31:00,0.95,14947.0,United Kingdom


## ต้องการนำตารางมาต่อกัน

สำหรับเนื้อหาในวันนี้ จะพิจารณาข้อมูลทั้ง 2 เดือน ดังนั้นจึงจำเป็นที่ต้องนำตัวแปร `data_jul` และ `data_aug` มาต่อกัน (concatenation)


### การเพิ่มคอลัมน์ที่เก็บค่าเดียวเหมือนกันหมด

ก่อนที่จะนำตารางมาต่อกัน เราจะเพิ่มคอลัมน์ที่ระบุว่า แต่ละตารางมาจาก Sheet ไหน ดังนี้

In [ ]:
data_jul['Sheet'] = 'July'    # สร้างคอลัมน์ใหม่ชื่อ Sheet ใน data_jul ที่มีค่าเท่ากับ July ทั้งหมด
data_aug['Sheet'] = 'August'  # สร้างคอลัมน์ใหม่ชื่อ Sheet ใน data_aug ที่มีค่าเท่ากับ August ทั้งหมด
data_aug.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
0,561904,22075,6 RIBBONS ELEGANT CHRISTMAS,96,2011-08-01 08:30:00,1.45,17941.0,United Kingdom,August
1,561904,85049E,SCANDINAVIAN REDS RIBBONS,156,2011-08-01 08:30:00,1.06,17941.0,United Kingdom,August
2,561905,21385,IVORY HANGING DECORATION HEART,24,2011-08-01 09:31:00,0.85,14947.0,United Kingdom,August
3,561905,84970L,SINGLE HEART ZINC T-LIGHT HOLDER,12,2011-08-01 09:31:00,0.95,14947.0,United Kingdom,August
4,561905,84970S,HANGING HEART ZINC T-LIGHT HOLDER,12,2011-08-01 09:31:00,0.85,14947.0,United Kingdom,August


### การนำตารางมาต่อกัน

การนำตารางมาต่อกัน ทำได้ด้วยคำสั่ง `pd.concat()` ซึ่งมีรูปแบบการใช้ ดังนี้

```python
new_data = pd.concat( tuple_ที่เก็บตัวแปรตาราง )
```

เช่น หากมีตาราง `A`, `B` และ `C` ที่ต้องการนำมาต่อกันตามลำดับ ก็ให้กำหนด `tuple_ที่เก็บตัวแปรตาราง = (A, B, C)` เป็นต้น โดย `Pandas` จะทำการนำคอลัมน์ของ  `A`, `B` และ `C` ที่ชื่อเหมือนกัน มาต่อกัน (ซึ่งหมายความว่า หากในตาราง มีลำดับคอลัมน์ไม่เหมือนกัน แต่ชื่อคอลัมน์ตรงกัน ตารางก็จะยังสามารถนำมาต่อกันได้อย่างถูกต้อง)

In [ ]:
data = pd.concat( (data_jul, data_aug))   # ต้องการเอา `data_aug` มาต่อท้าย `data_jul`
data.sample(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
32441,564818,22553,PLASTERS IN TIN SKULLS,1,2011-08-30 12:10:00,3.29,NaN,United Kingdom,August
38431,561870,22398,MAGNETS PACK OF 4 SWALLOWS,12,2011-07-31 11:45:00,0.39,14911.0,EIRE,July
36882,561658,22733,3D TRADITIONAL CHRISTMAS STICKERS,2,2011-07-28 16:06:00,1.25,12743.0,Unspecified,July
4629,559037,82583,NaN,10,2011-07-05 15:29:00,0.00,NaN,United Kingdom,July


สำหรับข้อมูลในวันนี้ ทั้ง `data_jul` และ `data_aug`  มีชื่อคอลัมน์เหมือนกัน จึงสามารถนำมาต่อกันได้โดยง่าย

อย่างไรก็ตาม `DataFrame` ที่ได้จากการต่อตาราง จะมี `index` ที่ซ้ำกัน เพราะว่า `index` ของทั้ง `data_jul` และ `data_aug` ต่างเริ่มนับจาก `0` ปัญหานี้ สามารถตรวจสอบได้ เช่น

In [ ]:
data[data.index == 555]  # พิมพ์แถวที่มี index เท่ากับ 555 ออกมา ซึ่งจะพบว่ามีซ้ำกัน 2 แถว (แต่ละแถวมาจากแต่ละ Sheet)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
555,558702,23146,TRIPLE HOOK ANTIQUE IVORY ROSE,2,2011-07-01 12:45:00,3.29,12748.0,United Kingdom,July
555,561968,23152,IVORY SWEETHEART WIRE LETTER RACK,2,2011-08-01 13:29:00,3.75,16332.0,United Kingdom,August


การที่ `index` มีค่าซ้ำกัน อาจทำให้เกิดความผิดพลาดในการคำนวณในอนาคต ดังนั้น หากไม่ต้องการให้ `pd.concat()` ใช้ `index` เดิม (ซึ่งสร้างอัตโนมัติมา) แต่ให้สร้าง `index` ใหม่ ก็ทำได้ด้วยการเพิ่มพารามิเตอร์ `ignore_index=True`

In [ ]:
data = pd.concat( (data_jul, data_aug), ignore_index=True)
data[data.index == 555]   # คราวนี้จะเห็นว่า index เท่ากับ 555 มีแถวเดียว

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
555,558702,23146,TRIPLE HOOK ANTIQUE IVORY ROSE,2,2011-07-01 12:45:00,3.29,12748.0,United Kingdom,July


**ลองทำ:**

- `ignore_index=True` ทำให้สูญเสีย `index` ของข้อมูลเดิม ซึ่งอาจเป็นปัญหาในกรณีที่ต้องการเข้าถึงค่า `index` เดิม ดังนั้น หากต้องการเก็บค่า `index` เดิมไว้ จะต้องทำอย่างไร (ใช้ความรู้จากคาบก่อน)

In [ ]:
# เฉลย
data_jul_reset_index = data_jul.reset_index()  # จะมีคอลัมน์ชื่อ index เพิ่มเข้ามา
data_aug_reset_index = data_aug.reset_index()
data = pd.concat( (data_jul_reset_index, data_aug_reset_index), ignore_index=True)
data.sample(4)

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
43431,3913,562264,21700,BIG DOUGHNUT FRIDGE MAGNETS,6,2011-08-03 16:14:00,1.38,NaN,United Kingdom,August
20979,20979,560272,22352,LUNCH BOX WITH CUTLERY RETROSPOT,6,2011-07-17 14:56:00,2.55,12691.0,France,July
57850,18332,563552,22171,3 HOOK PHOTO SHELF ANTIQUE WHITE,4,2011-08-17 13:06:00,8.50,17656.0,United Kingdom,August
21281,21281,560290,21930,JUMBO STORAGE BAG SKULLS,1,2011-07-18 10:17:00,4.96,NaN,United Kingdom,July


## การลบคอลัมน์

สมมติว่า ไม่ต้องการใช้คอลัมน์ `Sheet` ทีสร้างขึ้นอีกแล้ว และอยากจะลบออกจาก `DataFrame` เลย จะลบได้โดยใช้ฟังก์ชัน

```
DataFrame.drop(columns=...)
```
โดยที่ `columns` คือชื่อคอลัมน์หรือรายการชื่อคอลัมน์ที่ต้องการลบ

In [ ]:
data.drop(columns=['Sheet']) # จะเห็นว่าผลลัพธ์ที่ได้ คอลัมน์ Sheet หายไป

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,558638,84836,ZINC METAL HEART DECORATION,12,2011-07-01 08:16:00,1.25,16317.0,United Kingdom
1,1,558638,71459,HANGING JAM JAR T-LIGHT HOLDER,24,2011-07-01 08:16:00,0.85,16317.0,United Kingdom
2,2,558638,22784,LANTERN CREAM GAZEBO,3,2011-07-01 08:16:00,4.95,16317.0,United Kingdom
3,3,558638,23145,ZINC T-LIGHT HOLDER STAR LARGE,12,2011-07-01 08:16:00,0.95,16317.0,United Kingdom
4,4,558638,22674,FRENCH TOILET SIGN BLUE METAL,12,2011-07-01 08:16:00,1.25,16317.0,United Kingdom
...,...,...,...,...,...,...,...,...,...
74797,35279,C565077,23196,VINTAGE LEAF MAGNETIC NOTEPAD,-3,2011-08-31 17:31:00,1.45,17451.0,United Kingdom
74798,35280,C565077,22189,CREAM HEART CARD HOLDER,-1,2011-08-31 17:31:00,3.95,17451.0,United Kingdom
74799,35281,C565077,23239,SET OF 4 KNICK KNACK TINS POPPIES,-1,2011-08-31 17:31:00,4.15,17451.0,United Kingdom
74800,35282,C565077,23197,SKETCHBOOK MAGNETIC SHOPPING LIST,-12,2011-08-31 17:31:00,1.45,17451.0,United Kingdom


## เกร็ดการใช้ `Pandas` (`inplace=True`)

ฟังก์ชันที่แก้ไข `DataFrame`อย่างเช่นฟังก์ชัน `DataFrame.drop()` ฯลฯ มันไม่เขียนทับต้นฉบับเดิม ดังนั้นเวลาจะนำไปใช้ต่อ จึงต้อง assign ค่าให้ตัวแปรใหม่ หรือหากต้องการเขียนทับตัวแปรเดิม ในตัวอย่างก่อนหน้า ก็จะต้องเขียนว่า
```python
 data = data.drop(columns=['Sheet'])   # แบบที่ 1
```

หรืออีกทางเลือกหนึ่ง ซึ่ง `Pandas`  อำนวยความสะดวกให้ในกรณีที่ต้องการเขียนทับต้นฉบับ คือการกำหนดพารามิเตอร์ `inplace=True` ดังนี้
```python
 data.drop(columns=['Sheet'], inplace=True)   # แบบที่ 2
```

ซึ่งหากเขียนแบบนี้แล้ว ก็ไม่ต้องนำตัวแปรมารับค่าอีก ซึ่งหากทำดังโค้ดต่อไปนี้ ก็จะเกิดข้อผิดพลาด นำตัวแปร `data` ไปใช่ต่อไปไม่ได้
```python
# ผิดพลาด!! ทำแบบนี้แล้ว data ที่ได้จะเป็นค่า None
 data = data.drop(columns=['Sheet'], inplace=True)  
 print(data)  # จะได้ None
```
สาเหตุที่เป็นเช่นนี้ก็เพราะ
- `inplace=False` ทำให้ฟังก์ชัน `return` ตัวแปรที่ผ่านการแก้ไข แต่ต้นฉบับเดิมยังไม่เปลี่ยน
- `inplace=True` ทำให้ฟังก์ชันแก้ตัวแปรต้นฉบับ พร้อมกับ `return` ค่า `None`

**คำแนะนำ**
- เลือกใช้ตามความเหมาะสม

## ตรวจสอบข้อมูล `Quantity` และ `UnitPrice`

เนื่องจากข้อมูลที่ได้รับมา อาจมีความไม่ถูกต้องในข้อมูลอยู่ด้วย ยกตัวอย่างเช่น หากลองสุ่มแถวมาดูเรื่อย ๆ จะพบสิ่งผิดปกติ  บางแถวมี `Quantity` ติดลบ หรือแม้กระทั่ง `UnitPrice` ติดลบ ซึ่งเราไม่ทราบว่ามีที่มาอย่างไร ดังนี้ เราจะลองเข้าไปดูหัวข้อนี้

### มี `Quantity` น้อยกว่า `0 ` หรือ `UnitPrice` น้อยกว่า `0` หรือไม่

ก่อนอื่น จะสุ่มพิมพ์แถวที่มี `'Quantity' < 0` จะพบว่ามี `Quantity` ติดลบอยู่จริง

In [ ]:
data[data['Quantity'] < 0].sample(5)    # Quantity < 0 ได้อย่างไร?

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
24178,24178,C560540,23190,BUNDLE OF 3 SCHOOL EXERCISE BOOKS,-1,2011-07-19 12:26:00,1.65,12415.0,Australia,July
32043,32043,C561198,M,Manual,-35,2011-07-25 14:45:00,1.25,16161.0,United Kingdom,July
42516,2998,C562159,M,Manual,-12,2011-08-03 11:48:00,1.45,17389.0,United Kingdom,August
9537,9537,C559347,85159A,"BLACK TEA,COFFEE,SUGAR JARS",-24,2011-07-07 17:22:00,1.95,18072.0,United Kingdom,July
74671,35153,C565025,82494L,WOODEN FRAME ANTIQUE WHITE,-2,2011-08-31 16:32:00,2.95,13050.0,United Kingdom,August


**ลองทำ:** จงหาว่า `'UnitPrice'` ที่เป็นลบ มาทั้งหมดกี่แถว

In [ ]:
# เฉลย
data[data['UnitPrice'] < 0] # จะเห็น InvoiceNo มีตัว A นำหน้าด้วย

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
54080,14562,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,August
54081,14563,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,August


### `InvoiceNo` กับ `Quantity` และ `UnitPrice`

จากผลการพิมพ์แถวที่มี `'UnitPrice' < 0`จะพบความผิดปกติของข้อมูลที่ไม่สอดคล้องกับคำอธิบายข้อมูล (Data dictionary) ที่ผู้แจกจ่ายข้อมูลให้มา นี่เป็นปัญหาปกติที่ต้องเผชิญอย่างหลีกเลี่ยงไม่ได้ ก่อนที่จะสามารถทำข้อมูลไปใช้ประโยชน์ได้

Data dictionary บอกทราบว่า `InvoiceNo` เป็นเลข 6 หลัก ที่หากขึ้นต้นด้วย `'C'` จะหมายถึงการยกเลิกสินค้า แต่จากการพิมพ์แถวที่ `'UnitPrice' < 0`จะพบ  `InvoiceNo` ขึ้นต้นด้วย `'A'` ด้วย ดังนั้นในหัวข้อนี้จะเจาะลึกคอลัมน์ `InvoiceNo`


**คำถามที่ต้องการตอบ**

1. จะเข้าถึงแถวที่ค่าในคอลัมน์ `InvoiceNo` ขึ้นต้นด้วย `C` หรือ `A` ได้อย่างไร
2. ถ้า `InvoiceNo` ขึ้นต้นด้วย `C` แล้ว `Quantity` น้อยกว่า `0` เสมอ?

เพื่อที่จะได้คำตอบ จะทำเป็นขั้น ๆ ดังนี้

#### **1) การเข้าถึงแถวที่ค่าในคอลัมน์ `InvoiceNo` ขึ้นต้นด้วย `C` หรือ `A`**

***ตรวจสอบว่าคอลัมน์ `InvoiceNo` เก็บข้อมูลชนิดใด***



In [ ]:
data['InvoiceNo'].dtype    # จะใช้คำสั่ง data.info() จากคาบก่อนเพื่อตรวจดูก็ได้

dtype('O')

จะเห็นว่า เป็นชนิดแบบผสม (`dtype('O')` คือ `object`)

**แปลงชนิดข้อมูลของคอลัมน์ `InvoiceNo`**

การที่ข้อมูล `InvoiceNo` เป็นชนิด `object` จะยากแก่การตรวจสอบว่า แต่ละค่าขึ้นต้นด้วยอักษร `C` หรือไม่ ดังนั้นจึงจะแปลงชนิดข้อมูลในคอลัมน์นี้ ให้เป็นชนิด `string` เสียก่อน ด้วยคำสั่ง `Series.astype('string')` ดังนี้

In [ ]:
# หมายเหตุ: คอลัมน์อื่น ๆ ที่จำเป็นต้องแปลงข้อมูลให้ถูกต้องก็มีอีกหลายคอลัมน์ แต่ขอข้ามไปก่อน
data['InvoiceNo'] = data['InvoiceNo'].astype('string')  # แปลงแล้วเขียนทับลงคอลัมน์เดิม
data['InvoiceNo'].dtype

string[python]

**ตรวจสอบว่า `InvoiceNo` ขึ้นต้นด้วย `C` หรือไม่**

เมื่อคอลัมน์ `InvoiceNo` ถูกแปลงเป็นชนิด `string` แล้ว จะสามารถใช้ฟังก์ชันเกี่ยวกับ `string` ของ `Pandas` ได้อย่างอิสระ

การจะเข้าถึงสมาชิกแต่ละตัวของ `Series` ที่เก็บ `string` ได้นั้น จะต้องกระทำผ่านตัวช่วยที่เรียกว่า **String method** (`.str`) ดังรูปแบบ ต่อไปนี้
```python
Series.str.ฟังก์ชันสตริง
```

ซึ่งตัวอย่างของฟังก์ชัน ได้แก่

| ฟังก์ชันสตริง  | ตัวอย่างการใช้งาน |  คำอธิบาย |
|:---|:---|:--|
|`startswith()` |`Series.str.startswith('H')` | เช็คว่าอักษรตัวแรกเป็น `H` หรือไม่ |
|`endswith()` |`Series.str.endswith('y')` |  เช็คว่าอักษรตัวสุดท้ายเป็น `y` หรือไม่ |
|`lstrip()` |`Series.str.lstrip('ah')` | ไล่ลบตัวอักษร `a` หรือ `h` โดยเริ่มลบจากทางซ้ายไปขวา และลบจนกว่าจะเจออักษรที่ไม่ใช่ `a` หรือ `h`   |
| |`Series.str.lstrip()`   | ลบช่องว่างด้านซ้าย |
|`rstrip()` | `Series.str.rstrip()`|   คล้าย `lstrip()` แต่ไล่ลบเริ่มจากด้านขวา  |
|`strip()` |`Series.str.strip()` |  กระทำทั้ง `lstrip()` และ `rstrip()`  |
|`len()` |`Series.str.len()` |  นับจำนวนตัวอักษร  |

สำหรับตัวอย่างการใช้งาน String method ของ `Pandas` อื่น ๆ ศึกษาเพิ่มเติมได้จาก [ลิงค์](https://pandas.pydata.org/docs/user_guide/text.html#string-methods) และ[ตารางสรุปฟังก์ชัน](https://pandas.pydata.org/docs/user_guide/text.html#method-summary)



ในกรณีนี้ เราต้องการตรวจสอบว่า `InvoiceNo` ขึ้นต้นด้วยอักษร `C` หรือไม่ จึงใช้ `startswith()` ดังนี้

In [ ]:
flag_c = data['InvoiceNo'].str.startswith('C')  # True คือขึ้นต้นด้วย C ส่วน False ไม่ขึ้นต้นด้วย C

In [ ]:
data_c = data[flag_c]
print(f'Invoice ขึ้นต้นด้วย C มี {len(data_c)} แถว')
data_c.sample(5)

Invoice ขึ้นต้นด้วย C มี 1353 แถว


,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
63791,24273,C564138,21218,RED SPOTTY BISCUIT TIN,-3,2011-08-23 11:36:00,3.25,12753.0,Japan,August
67048,27530,C564424,15039,SANDALWOOD FAN,-20,2011-08-25 11:10:00,0.85,12955.0,United Kingdom,August
47914,8396,C562620,22801,ANTIQUE GLASS PEDESTAL BOWL,-1,2011-08-08 12:34:00,3.75,13089.0,United Kingdom,August
4037,4037,C558969,22728,ALARM CLOCK BAKELIKE PINK,-1,2011-07-05 11:01:00,3.75,16729.0,United Kingdom,July
4526,4526,C559030,21868,POTTING SHED TEA MUG,-2,2011-07-05 14:51:00,1.25,16571.0,United Kingdom,July


และในลักษณะเดียวกัน สามารถเข้าถึงแถวที่ขึ้นต้นด้วย `'A'`

In [ ]:
flag_a = data['InvoiceNo'].str.startswith('A')
data_a = data[flag_a]
print(f'Invoice ขึ้นต้นด้วย A มี {len(data_a)} แถว')
data_a

Invoice ขึ้นต้นด้วย A มี 3 แถว


,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
54079,14561,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom,August
54080,14562,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,August
54081,14563,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,August


#### 2) ถ้า `InvoiceNo` ขึ้นต้นด้วย `C` แล้ว `Quantity` ติดลบเสมอ?

นับจำนวนแถวของ `InvoiceNo` ที่เป็นการยกเลิก และมี `Quantity` น้อยกว่า `0`


In [ ]:
flags = (data_c['Quantity'] < 0) # ในบรรดา Invoice ที่ขึ้นต้นด้วย C
len(data_c[flags])    # Invoice ขึ้นต้นด้วย C และ Quantity < 0 มี 1353 แถว ซึ่งเท่ากับจำนวนแถวทั้งหมดของ data_c

1353

In [ ]:
flags = (data_c['Quantity'] > 0)
len(data_c[flags])   # ไม่มี Invoice ขึ้นต้นด้วย C ที่มี Quantity > 0

0

ดังนั้นจากข้อมูลที่มี จึงสรุปได้ว่า ถ้า `InvoiceNo` ขึ้นต้นด้วย `C` แล้ว `Quantity` จะน้อยกว่า `0` เสมอ

นอกจากนี้ เรายังพบอีกว่า คอลัมน์ `Quantity` ไม่มีค่าที่เท่ากับ `0` เลย

In [ ]:
flags = (data['Quantity'] == 0) # เช็คข้อมูลทั้งหมด
len(data[flags])   # ไม่พบว่าในชุดข้อมูลทั้งหมดมีการบันทึกค่า Quantity == 0

0

#### **แบบฝึกหัด**

จงเติมตารางต่อไปนี้ให้สมบูรณ์ (ช่องที่เป็นเครื่องหมาย `?` เป็นค่าอะไร)

*หมายเหตุ:* เราทราบก่อนหน้าแล้วว่า `Quantity` ไม่เคยเท่ากับ `0`

| `InvoiceNo`  | `Quantity` | `UnitPrice` | จำนวนแถว | สมมติฐาน |
|:--|:--|:--|--:| :--|   
| `C` | `> 0`| `> 0` | 0 | |
| `C` | `> 0`| `== 0` | 0 | |
| `C` | `> 0` | `< 0` | 0 | |
| `C` | `< 0` | `> 0` | 1353 |  การคืนสินค้า จะใส่ `Quantity` ติดลบ  แต่ `UnitPrice` อาจจะเป็น `UnitPrice` ที่ซื้อไป จึงเป็นบวก |
| `C` | `< 0` | `== 0` | 0 |  |
| `C` | `< 0` | `< 0` | 0 | |
| not `C` |`> 0` |`> 0` | 73128 | รายการขายออกไป จึงมีแต่ค่าบวก (แต่มีรายการที่ `InvoiceNo` ขึ้นต้นด้วย `A` อยู่ 1 รายการ ) |
| not `C` |`> 0` |`== 0` |  **?** | หลากหลายเหตุผลที่ไม่อาจทราบได้ เช่นอาจเป็นสินค้าที่คิดว่าเสียหาย แต่เพิ่งเจอใน stock, อาจลงเลขผิด ฯลฯ  |
| not `C` |`> 0` |`< 0` | 2 | รายการที่ `InvoiceNo` ขึ้นต้นด้วย `A` |
| not `C` |`< 0` |`> 0` | 0 | |
| not `C` |`< 0` |`== 0` | **?**   | หลากหลายเหตุผลที่ไม่อาจทราบได้ เช่นอาจเป็นสินค้าที่เสียหาย ที่ไม่ได้ขายออกไป, อาจลงเลขผิด ฯลฯ  |
| not `C` |`< 0` |`< 0` | 0 | |
| | | รวม | 74802 | |

  
  

In [ ]:
# เฉลย
ans1 = len(data[~data['InvoiceNo'].str.startswith('C') & (data['Quantity'] > 0) & (data['UnitPrice'] == 0)])
ans2 = len(data[~data['InvoiceNo'].str.startswith('C') & (data['Quantity'] < 0) & (data['UnitPrice'] == 0)])
print(f'คำตอบคือ {ans1} กับ {ans2} ตามลำดับ')

คำตอบคือ 152 กับ 167 ตามลำดับ


## การแปลงชนิดข้อมูลให้ถูกประเภท

ข้อมูลแต่ละคอลัมน์ใน `DataFrame` ถูกกำหนดชนิดข้อมูลโดยอัตโนมัติ ซึ่งอาจมีความผิดพลาดได้ ดังนั้น จึงเป็นหน้าที่ของนักวิเคราห์ข้อมูลที่จำต้องกำหนดให้ถูกต้องแต่เนิ่น ๆ


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74802 entries, 0 to 74801
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   index        74802 non-null  int64         
 1   InvoiceNo    74802 non-null  string        
 2   StockCode    74802 non-null  object        
 3   Description  74565 non-null  object        
 4   Quantity     74802 non-null  int64         
 5   InvoiceDate  74802 non-null  datetime64[ns]
 6   UnitPrice    74802 non-null  float64       
 7   CustomerID   55164 non-null  float64       
 8   Country      74802 non-null  object        
 9   Sheet        74802 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4), string(1)
memory usage: 5.7+ MB


จาก `data.info()` จะพบว่า `InvoiceNo` ถูกแปลงไว้ถูกต้องแล้วจากหัวข้อที่แล้ว

ส่วนคอลัมน์อื่น ๆ จะแปลงด้วยโค้ดด้านล่างนี้ โดยที่

- `'Int64'` คือตัวแปรจำนวนเต็ม (integer) ขนาด 64 บิต ที่รองรับการเก็บค่า `NaN`

**คำถาม:** ขอให้นักศึกษาอภิปรายประเด็นต่อไปนี้

1. เดิม `'StockCode'` เป็นประเภท `object` นั้นไม่ดีอย่างไร แล้วทำไมไม่แปลงเป็น `int`
2. ทำไมไม่แปลง `'CustomerID'` เป็น `int` หรือ?



In [ ]:
data['StockCode'] = data['StockCode'].astype('string')    # ทำไมไม่เป็น .astype(int)
data['Description'] = data['Description'].astype('string')
data['CustomerID'] = data['CustomerID'].astype('Int64')   # ทำไมไม่เป็น .astype(int)
data['Country'] = data['Country'].astype('string')

**เฉลย:**

1. เพราะ `StockCode` ไม่ใช่ตัวเลขเท่านั้น แต่มีรหัสที่เป็นตัวอักษรด้วย ดังนั้นจึงแปลงเป็น `int` ไม่ได้อยู่แล้ว ส่วนเหตุผลที่ไม่เก็บเป็น `object` ก็เช่น ใช้ String method ไม่ได้ หรือในแง่ของการใช้งาน การที่มีทั้ง `int` และ `string` ปนกัน ก็อาจสับสนเวลาใช้ เช่น จะใช้ `StockCode == 55555` กับ `StockCode == '55555'` กันแน่ เป็นต้น
2. เพราะ `CustomerID` มี `NaN` ซึ่ง ตัวแปร `int` ธรรมดาของ `Python` ไม่รองรับ

## เกร็ด: การทำซ้ำข้อมูล `DataFrame` ด้วย `.copy()`

เนื่องจากในหัวข้อต่อ ๆ ไป จะมีการแก้ไข `DataFrame` ไปเรื่อย ๆ ดังนั้น จึงขอคัดลอกตัวแปร `data` ไว้อีกฉบับหนึ่งในชื่อ `original_data` ดังนี้

In [ ]:
original_data = data.copy()

โดยการทดลองต่าง ๆ ให้ใช้ตัวแปร `data` และในกรณีที่ต้องการโหลดข้อมูลต้นฉบับมาใหม่ ให้ใช้คำสั่ง

```
data = original_data.copy()
```

ซึ่งจะเป็นการโหลดต้นฉบับ กลับมาให้ตัวแปร `data`

### ทำไมต้องใช้ `.copy()` ?

อย่างไรก็ตาม อาจมีคำถามว่า ทำไมจึงต้องใช้ `.copy()`  เหตุใดไม่เขียนโค้ด ดังนี้

```python
new_data = data   # ไม่มี .copy()
```
คำตอบก็คือ การเขียนแบบนี้ จะได้ตัวแปร `new_data` ที่หมายถึงข้อมูลที่เก็บใน RAM ที่เดียวกับของตัวแปร `data` แม้ชื่อจะต่างกัน (หรือพูดง่าย ๆ ว่า `new_data` เป็นชื่อเล่น ของ `data`) ดังนั้น หากมีการแก้ไข `data` หรือ `new_data` ตัวใดตัวหนึ่ง ด้วยคำสั่งของ `Pandas` บางคำสั่ง อาจทำให้ตัวแปรอีกตัวเปลี่ยนตามด้วย ซึ่งจะไม่ตรงกับจุดประสงค์ของเราที่ต้องการสำรองค่าตัวแปรไว้

เพื่อทำความเข้าใจ ขอให้นึกศึกษา พิจารณา 3 กรณี ต่อไปนี้

**กรณีที่ 1** จะเห็นว่า เมื่อไม่ใช้ `.copy()` เมื่อมีการแก้ไข `new_data` กลับทำให้ `data` เปลี่ยนตาม (จำนวนแถวลดลง)

**แบบนี้ต้องระวัง!!! อาจส่งผลให้การคำนวณต่อจากนั้นผิดพลาด**

เพราะ `new_data` เป็นชื่อเล่นของ `data` ที่หมายถึงข้อมูลที่เก็บไว้ใน RAM ที่เดียวกัน

In [ ]:
data = original_data.copy()  # โหลดข้อมูลต้นฉลับ
new_data = data              # กำหนดค่าโดยไม่มี .copy()
print(f'ก่อน: data.shape={data.shape}, new_data.shape={new_data.shape}')
new_data.dropna(inplace=True)  # ใช้ inplace=True (.dropna อยู่ในหัวข้อ Q3.3)
print(f'หลัง: data.shape={data.shape}, new_data.shape={new_data.shape}')

ก่อน: data.shape=(74802, 10), new_data.shape=(74802, 10)
หลัง: data.shape=(55164, 10), new_data.shape=(55164, 10)


**กรณีที่ 2** จะเห็นว่า เมื่อใช้ `copy()` เมื่อมีการแก้ไข `new_data` ไม่ทำให้ `data` เปลี่ยนตาม

**แบบนี้ OK**

เพราะ `copy()` ทำให้ `new_data` กับ `data` เก็บข้อมูลใน RAM คนละที่

In [ ]:
data = original_data.copy()   # โหลดข้อมูลต้นฉลับ
new_data = data.copy()        # กำหนดค่าโดยมี .copy()
print(f'ก่อน: data.shape={data.shape}, new_data.shape={new_data.shape}')
new_data.dropna(inplace=True)  # ใช้ inplace=True
print(f'หลัง: data.shape={data.shape}, new_data.shape={new_data.shape}')

ก่อน: data.shape=(74802, 10), new_data.shape=(74802, 10)
หลัง: data.shape=(74802, 10), new_data.shape=(55164, 10)


**กรณีที่ 3**  จะเห็นว่า ไม่ใช้ `copy()` เมื่อมีการแก้ไข `new_data` กลับทำให้ `data` เปลี่ยนตาม (จำนวนแถวลดลง)

**แบบนี้ก็ OK แต่จะเกิดข้อผิดพลาดภายหลังหากเผลอใช้ `inplace=True`**

สาเหตุที่แบบนี้ไม่เกิดปัญหา แม้ว่าตอนแรก `new_data` กับ `data` จะหมายถึงข้อมูลใน RAM ที่เดียวกัน แต่คำสั่ง `new_data.dropna()` ให้ค่าที่ RAM ตำแหน่งใหม่ ซึ่งเราก็ไปตั้งชื่อมันว่า `new_data`

In [ ]:
data = original_data.copy()    # โหลดข้อมูลต้นฉลับ
new_data = data                # กำหนดค่าโดยไม่มี .copy()
print(f'ก่อน: data.shape={data.shape}, new_data.shape={new_data.shape}')
new_data = new_data.dropna()   # ไม่ใช้ inplace=True
print(f'หลัง: data.shape={data.shape}, new_data.shape={new_data.shape}')

ก่อน: data.shape=(74802, 10), new_data.shape=(74802, 10)
หลัง: data.shape=(74802, 10), new_data.shape=(55164, 10)


## ข้อมูลที่ขาดหาย (Missing value)

ข้อมูลที่ขาดหาย ก็คือข้อมูลที่ว่างเปล่าตั้งแต่ในไฟล์ CSV หรือ Excel ซึ่งพออ่านเข้ามาใน `Pandas` จะถูกแทนที่ด้วยค่า `NaN` หรือ `<NA>` (หรือเรียกว่า null) [ดูคู่มือ](https://pandas.pydata.org/docs/user_guide/missing_data.html)

ข้อมูลขายของออนไลน์ในคาบนี้มีความยากกว่าข้อมูลการให้ทิปในคาบก่อนอยู่มาก ส่วนหนึ่งก็มาจากข้อมูลที่ไม่สมบูรณ์ การตรวจสอบข้อมูลที่ขาดหายไป ดูได้จาก `DataFrame.info()` เหมือนเช่นในคาบผ่านมา

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74802 entries, 0 to 74801
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   index        74802 non-null  int64         
 1   InvoiceNo    74802 non-null  string        
 2   StockCode    74802 non-null  string        
 3   Description  74565 non-null  string        
 4   Quantity     74802 non-null  int64         
 5   InvoiceDate  74802 non-null  datetime64[ns]
 6   UnitPrice    74802 non-null  float64       
 7   CustomerID   55164 non-null  Int64         
 8   Country      74802 non-null  string        
 9   Sheet        74802 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(2), object(1), string(4)
memory usage: 5.8+ MB


หรืออาจคำนวณเองได้จากฟังก์ชัน `DataFrame.isna()` ซึ่งจะให้ผลลัพธ์เป็น `DataFrame` ที่เก็บตัวแปร `bool` ทั้งหมด  

In [ ]:
flags = data.isna()
flags.sample(5)  # False แปลว่ามีข้อมูล แต่ถ้า True แปลว่า missing value

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
39194,False,False,False,False,False,False,False,False,False,False
42562,False,False,False,False,False,False,False,False,False,False
18450,False,False,False,False,False,False,False,True,False,False
70279,False,False,False,False,False,False,False,False,False,False
71717,False,False,False,False,False,False,False,True,False,False


และเนื่องจาก `False` สามาถแปลงเป็นค่า `0` และ `True` สามารถแปลงเป็นค่า `1` ได้อัตโนมัติ ดังนั้น จึงใช้ฟังก์ชัน `DataFrame.sum()` ในการนับจำนวน missing value ได้

In [ ]:
flags.sum()

index              0
InvoiceNo          0
StockCode          0
Description      237
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     19638
Country            0
Sheet              0
dtype: int64

จากรายงาน จะเห็นว่าข้อมูลทั้งหมดมี 74,802 แถว แต่ `Description` มีข้อมูลขาดหายไป 237 ตำแหน่ง ส่วน `CustomerID` ขาดหายไป 19638 ตำแหน่ง

หากจะลองสุ่มตรวจดูแถวที่มี `CustomerID` เป็น `NaN` ก็สามารถทำได้ ดังนี้

In [ ]:
data[data['CustomerID'].isna()].head()

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
139,139,558663,23574,<NA>,100,2011-07-01 10:44:00,0.00,<NA>,United Kingdom,July
171,171,558680,20711,JUMBO BAG TOYS,1,2011-07-01 11:19:00,4.13,<NA>,United Kingdom,July
172,172,558680,21116,OWL DOORSTOP,1,2011-07-01 11:19:00,8.29,<NA>,United Kingdom,July
173,173,558680,21166,COOK WITH WINE METAL SIGN,1,2011-07-01 11:19:00,4.13,<NA>,United Kingdom,July
174,174,558680,21175,GIN + TONIC DIET METAL SIGN,1,2011-07-01 11:19:00,4.96,<NA>,United Kingdom,July


**ลองดู:** นอกจากฟังก์ชัน `.isna()` แล้ว ยังมีอีกฟังก์ชันคือ `.notna()` ซึ่งทำงานในลักษณะกันข้าม ขอให้ทดลองด้วยตนเองว่า `.notna()` ใช้แล้วให้ผลอย่างไร

### ถ้าไม่อยากมองข้าม `NaN` ต้องกำหนด `dropna=False`

โดย default นั้น ฟังก์ชันบางฟังก์ชันของ `Pandas` จะมองข้ามค่า `NaN` เช่น ฟังก์ชัน

- `Series.value_counts()` (เนื้อหาคาบที่ผ่านมา)
- `Series.nunique()`  (เนื้อหาคาบที่ผ่านมา)
- `DataFrame.groupby()`  (ยังไม่ได้สอน)

ดังนั้นจึงเป็นสิ่งที่ควรระวังหากจำเป็นจำต้องสนใจค่า `NaN` ด้วย จะต้องเรียกใช้ โดยกำหนด

- `Series.value_counts(dropna=False)`
- `Series.nunique(dropna=False)`  
- `DataFrame.groupby(..., dropna=False)`

และเพื่อเป็นตัวอย่างต่อไป จะขอยกตัวอย่างเฉพาะรายการสินค้า `StockCode == '21116'` ซึ่งคือ `OWL DOORSTOP`

In [ ]:
doorstop = data[data['StockCode'] == '21116']
print(f'จำนวนแถวเท่ากับ {len(doorstop)} แถว')
doorstop

จำนวนแถวเท่ากับ 32 แถว


,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
172,172,558680,21116,OWL DOORSTOP,1,2011-07-01 11:19:00,8.29,<NA>,United Kingdom,July
3314,3314,558888,21116,OWL DOORSTOP,2,2011-07-04 15:29:00,4.95,14056,United Kingdom,July
5183,5183,559052,21116,OWL DOORSTOP,1,2011-07-05 16:53:00,8.29,<NA>,United Kingdom,July
7510,7510,559163,21116,OWL DOORSTOP,1,2011-07-06 16:33:00,8.29,<NA>,United Kingdom,July
8856,8856,559337,21116,OWL DOORSTOP,1,2011-07-07 16:27:00,8.29,<NA>,United Kingdom,July
18170,18170,560078,21116,OWL DOORSTOP,3,2011-07-14 16:12:00,4.95,16040,United Kingdom,July
18298,18298,560090,21116,OWL DOORSTOP,1,2011-07-14 16:46:00,8.29,<NA>,United Kingdom,July
21311,21311,560291,21116,OWL DOORSTOP,1,2011-07-18 10:17:00,8.29,<NA>,United Kingdom,July
25086,25086,560603,21116,OWL DOORSTOP,3,2011-07-20 08:59:00,4.95,18180,United Kingdom,July
28459,28459,560901,21116,OWL DOORSTOP,3,2011-07-21 17:39:00,4.95,12476,Germany,July


In [ ]:
doorstop['CustomerID'].value_counts()  # มองข้าม NaN

14056    2
16040    1
18180    1
12476    1
12540    1
16609    1
12828    1
13023    1
16764    1
16206    1
16076    1
18143    1
12621    1
18061    1
13013    1
Name: CustomerID, dtype: Int64

In [ ]:
doorstop['CustomerID'].value_counts(dropna=False)  # จะเห็นว่านัง NaN ได้ 16 ค่า

<NA>     16
14056     2
16040     1
18180     1
12476     1
12540     1
16609     1
12828     1
13023     1
16764     1
16206     1
16076     1
18143     1
12621     1
18061     1
13013     1
Name: CustomerID, dtype: Int64

### เติมข้อมูลที่ขาดหายด้วยค่าคงที่

Missing value สามารถเติมได้โดยใช้คำสั่ง `Series.fillna()` หรือ `DataFrame.fillna()`

In [ ]:
data['Description'] = data['Description'].fillna('ไม่ทราบชื่อ')
data[data['Description'] == 'ไม่ทราบชื่อ']

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
139,139,558663,23574,ไม่ทราบชื่อ,100,2011-07-01 10:44:00,0.0,<NA>,United Kingdom,July
870,870,558724,35966,ไม่ทราบชื่อ,14,2011-07-01 14:30:00,0.0,<NA>,United Kingdom,July
879,879,558728,20826,ไม่ทราบชื่อ,-1,2011-07-01 14:50:00,0.0,<NA>,United Kingdom,July
950,950,558732,47341A,ไม่ทราบชื่อ,-1,2011-07-01 14:55:00,0.0,<NA>,United Kingdom,July
998,998,558741,37444C,ไม่ทราบชื่อ,-1,2011-07-01 15:52:00,0.0,<NA>,United Kingdom,July
...,...,...,...,...,...,...,...,...,...,...
74718,35200,565063,21175,ไม่ทราบชื่อ,-18,2011-08-31 17:15:00,0.0,<NA>,United Kingdom,August
74790,35272,565069,20665,ไม่ทราบชื่อ,-49,2011-08-31 17:18:00,0.0,<NA>,United Kingdom,August
74791,35273,565070,85032D,ไม่ทราบชื่อ,171,2011-08-31 17:18:00,0.0,<NA>,United Kingdom,August
74792,35274,565071,84754,ไม่ทราบชื่อ,-242,2011-08-31 17:21:00,0.0,<NA>,United Kingdom,August


### ไม่รู้จะเติม `NaN` ด้วยอะไร ก็ลบซะเลย

ก่อนอื่น เนื่องจาก `data` ถูกเปลี่ยนค่า `NaN` ไปในข้อ Q3.2 ดังนั้น จึงขอโหลดต้นฉบับกลับมา

In [ ]:
data = original_data.copy()

สามารถใช้คำสั่ง `DataFrame.dropna()` ในการลบแถว ซึ่งมีพารามิเตอร์ ได้แก่

- `how='any'` หมายถึง ถ้ามี `NaN` แค่ตัวเดียวในแถวนั้น ก็ให้ลบทั้งแถว หรือ `how='all'` หมายถึง ต้องมี `NaN` ทุกคอลัมน์ จึงจะลบได้ โดย `how='any'` เป็นค่า default  
- `subset=` รายการของชื่อคอลัมน์ที่จะให้พิจารณาว่ามี `NaN` หรือไม่ โดย `how`จะพิจารณาเฉพาะคอลัมน์ที่ระบุไว้ใน `subset`
- `axis=0` หรือ `axis=1`
  - หากต้องการพิจารณา `DataFrame` เป็นรายแถว ให้กำหนด `axis=0` (default)
  - หากต้องการพิจารณา `DataFrame` เป็นรายคอลัมน์ ให้กำหนด `axis=1` (ซึ่งกรณีนี้ `how=` จะพิจารณา `NaN` ของคอลัมน์หนึ่ง ๆ)
- `inplace=True` หรือ `inplace=False` หมายถึงให้เขียนทับต้นฉบับหรือไม่  

ขอให้นักศึกษาพิจารณาเปรียบเทียบการใช้งาน จากตัวอย่างต่อไปนี้

In [ ]:
print(f'จำนวนแถว {len(data)} แถว (ก่อน dropna)')
no_nan = data.dropna(how='any')
print(f'จำนวนแถวที่เหลือ {len(no_nan)} แถว (หลัง dropna ด้วย how=any)')
no_nan.isna().sum()  # แถวที่มี NaN ตัวเดียวขึ้นไป ถูกลบทั้งหมด

จำนวนแถว 74802 แถว (ก่อน dropna)
จำนวนแถวที่เหลือ 55164 แถว (หลัง dropna ด้วย how=any)


index          0
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
Sheet          0
dtype: int64

In [ ]:
print(f'จำนวนแถว {len(data)} แถว (ก่อน dropna)')
no_nan = data.dropna(how='any', subset=['Description','Quantity'])
print(f'จำนวนแถวที่เหลือ {len(no_nan)} แถว (หลัง dropna ด้วย how=any, subset=[Description, Quantity])')
no_nan.isna().sum()  # แถวที่คอลัมน์ Description และ Quantity มี NaN ตั้งแต่ 1 ตัวขึ้นไป จะถูกลบทั้งหมด

จำนวนแถว 74802 แถว (ก่อน dropna)
จำนวนแถวที่เหลือ 74565 แถว (หลัง dropna ด้วย how=any, subset=[Description, Quantity])


index              0
InvoiceNo          0
StockCode          0
Description        0
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     19401
Country            0
Sheet              0
dtype: int64

In [ ]:
print(f'จำนวนคอลัมน์ {data.shape[1]} คอลัมน์ (ก่อน dropna)')
no_nan = data.dropna(how='any', axis=1)
print(f'จำนวนคอลัมน์ที่เหลือ {no_nan.shape[1]} คอลัมน์ (หลัง dropna ด้วย how=any)')
no_nan.isna().sum()  # คอลัมน์ที่มี NaN ตัวเดียวขึ้นไป ถูกลบทั้งหมด (CustomerID กับ Description ถูกลบไป)

จำนวนคอลัมน์ 10 คอลัมน์ (ก่อน dropna)
จำนวนคอลัมน์ที่เหลือ 8 คอลัมน์ (หลัง dropna ด้วย how=any)


index          0
InvoiceNo      0
StockCode      0
Quantity       0
InvoiceDate    0
UnitPrice      0
Country        0
Sheet          0
dtype: int64

## Groupby and Aggregation

หากต้องการแบ่งส่วนของตารางเป็นกลุ่มย่อย ๆ ตามค่าของคอลัมน์ ๆ หนึ่ง (หรืออาจหลายคอลัมน์) สามารถทำได้ง่ายโดยใช้ฟังก์ชัน `DataFrame.groupby()` [ดูคู่มือ](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) และสามารถคำนวณค่าสถิติภาพรวมของแต่ละกลุ่มย่อย ด้วยการทำ aggregration (`.agg()`)

ยกตัวอย่างเช่น จากข้อมูล `data` ที่มี หากต้องการแบ่งกลุ่มย่อยตามค่าของคอลัมน์ `InvoiceNo` จะทำได้ ดังนี้

In [ ]:
inv_group = data.groupby('InvoiceNo', dropna=False)
inv_group

ตัวแปร `inv_group` ที่ได้เป็นชนิดข้อมูล `DataFrameGroupBy` ของ `Pandas`

การนำกลุ่มที่แบ่งขึ้นไปใช้ประโยชนในเบื้องต้น ได้แก่

### จะเข้าถึงกลุ่มย่อยได้อย่างไร

คำตอบคือ การเข้าได้โดยใช้คำสั่งดังนี้
```python
inv_group.get_group('ค่า InvoiceNo ที่ต้องการ')
```

เช่น จะเข้าถึงกลุ่มที่ `InvoiceNo` มีค่า `'558888'`

In [ ]:
inv_group.get_group('558888')

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sheet
3253,3253,558888,22902,TOTE BAG I LOVE LONDON,2,2011-07-04 15:29:00,2.10,14056,United Kingdom,July
3254,3254,558888,21577,SAVE THE PLANET COTTON TOTE BAG,3,2011-07-04 15:29:00,2.25,14056,United Kingdom,July
3255,3255,558888,23050,RECYCLED ACAPULCO MAT GREEN,5,2011-07-04 15:29:00,8.25,14056,United Kingdom,July
3256,3256,558888,23049,RECYCLED ACAPULCO MAT RED,3,2011-07-04 15:29:00,8.25,14056,United Kingdom,July
3257,3257,558888,23052,RECYCLED ACAPULCO MAT TURQUOISE,5,2011-07-04 15:29:00,8.25,14056,United Kingdom,July
...,...,...,...,...,...,...,...,...,...,...
3335,3335,558888,21888,BINGO SET,1,2011-07-04 15:29:00,3.75,14056,United Kingdom,July
3336,3336,558888,23207,LUNCH BAG ALPHABET DESIGN,6,2011-07-04 15:29:00,1.65,14056,United Kingdom,July
3337,3337,558888,23205,CHARLOTTE BAG VINTAGE ALPHABET,1,2011-07-04 15:29:00,0.85,14056,United Kingdom,July
3338,3338,558888,21912,VINTAGE SNAKES & LADDERS,1,2011-07-04 15:29:00,3.75,14056,United Kingdom,July


ซึ่งจะพบว่าตารางมี 87 แถว และคอลัมน์ `InvoiceNo` มีค่า `'558888'` เหมือนกันหมด

### การทำข้อมูลภาพรวมจากกลุ่มย่อย

การยุบคอลัมน์ในกลุ่มย่อยแต่ละกลุ่มให้เหลือเพียงค่าเดียว เช่น การหาค่าเฉลี่ย ค่าสูงสุด หรือจำนวนแถวของคอลัมน์ เป็นต้น เรียกว่าการทำ aggregration ซึ่งสามารถทำโดยใช้คำสั่ง

`group.sum()`, `group.min()` , `group.max()`, .... ฯลฯ หรือ `group.agg(['sum','min','max', ...])`

ซึ่งเป็นคำสั่งเดียวกับที่ได้ศึกษาในคาบที่ผ่านมา  (แต่เป็นคาบนั้นเป็นการหาค่าสถิติของทั้งตาราง)
ยกตัวอย่างเช่น

In [ ]:
inv_group.max(numeric_only=True)  # กำหนด numeric_only=True เพื่อที่จะคำนวณ sum เฉพาะคอลัมน์ที่เป็นตัวเลข

,index,Quantity,UnitPrice,CustomerID
InvoiceNo,,,,
558638,23,50,12.75,16317
558639,36,20,7.95,13492
558640,44,25,50.00,14911
558641,73,24,10.95,17865
558642,94,32,12.75,17667
...,...,...,...,...
C565044,35163,-113,7.65,12931
C565050,35172,-1,4.95,12381
C565075,35277,-1,38.39,13408


จะเห็นว่า ผลลัพธ์ของ aggregration ด้วย `.max()` นั้น จะได้ `DataFrame` (หรืออาจเป็น `Series` กรณีข้อมูลมีคอลัมน์เดียว) ที่แต่ละแถวแสดงค่าสูงสุดของแต่ละกลุ่ม

ทั้งนี้ สามารถเลือกคอลัมน์ที่ต้องการทำ aggregration ได้เหมือนกับการเข้าถึงคอลัมน์ของ `DataFrame` ดังเช่น

In [ ]:
inv_group[['UnitPrice','Quantity']].max()  # สามารถเลือกเฉพาะคอลัมน์ได้

,UnitPrice,Quantity
InvoiceNo,,
558638,12.75,50
558639,7.95,20
558640,50.00,25
558641,10.95,24
558642,12.75,32
...,...,...
C565044,7.65,-113
C565050,4.95,-1
C565075,38.39,-1


In [ ]:
inv_group[['UnitPrice','Quantity']].agg(['max','min'])

UnitPrice        Quantity     
                max    min      max  min
InvoiceNo                               
558638        12.75   0.42       50    2
558639         7.95   1.65       20    2
558640        50.00   0.42       25    1
558641        10.95   0.55       24    2
558642        12.75   0.42       32    1
...             ...    ...      ...  ...
C565044        7.65   1.27     -113 -318
C565050        4.95   1.25       -1  -12
C565075       38.39  36.30       -1   -1
C565077        4.15   1.45       -1  -12
C565078       57.60  57.60       -1   -1

[3664 rows x 4 columns]

### จะทราบได้อย่างไรว่า `group` ที่ได้มีกี่กลุ่ม อะไรบ้าง

คำตอบคือ สามารถนับจำนวนกลุ่มโดยใช้ `.ngroups` และสามารถเข้าถึงชื่อกลุ่มทั้งหมดได้จาก `.groups.keys()` ดังตัวอย่างต่อไปนี้

**1) นับจำนวนกลุ่ม**

In [ ]:
inv_group.ngroups

3664

จะเห็นว่ามีทั้งหมด 3664 กลุ่ม ซึ่งตรวจสอบด้วยคำสั่ง

In [ ]:
data['InvoiceNo'].nunique(dropna=False)

3664

**2) รายชื่อกลุ่มทั้งหมดที่เป็นไปได้**

```
inv_group.groups.keys()   # ให้นักศึกษาลองใช้ดู
```

## เกี่ยวกับข้อมูลวันที่ (`datetime`)

### การแปลง `string` เป็น `datetime`

สมมติว่ามี `string` ที่เก็บค่าวันที่ ดังนี้

- `start_date = '20/12/2023'`
- `end_date = '25/12/2023'`

เราคาดหวังว่า เมื่อเปรียบเทียบ `start_date < end_date` แล้วจะต้องได้ค่า `True` แต่ทว่าการกระทำเช่นนี้ไม่สามารถทำได้โดยง่ายระหว่างตัวแปรประเภท `string`

ด้วยเหตุนี้ จึงเกิดตัวแปรประเภท `datetime` ขึ้น ซึ่งวิธีสร้างตัวแปร `datetime` วิธีหนึ่ง ก็คือการแปลงจาก `string` ให้เป็น `datetime` โดยใช้คำสั่งต่อไปนี้

```python
pd.to_datetime(สตริงที่ต้องการแปลง, format=รูปแบบของวันเวลา)
```



ยกตัวอย่างเช่น


In [ ]:
birth_time = '19-Mar-98 15:34:56'  # = 19 March 1998 15:34:56
birth_time = pd.to_datetime(birth_time, format='%d-%b-%y %H:%M:%S')
birth_time

Timestamp('1998-03-19 15:34:56')

In [ ]:
start_date = '20/12/2023'
end_date = 'December 25, 2023'

start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
end_date = pd.to_datetime(end_date, format='%B %d, %Y')
print(start_date)
print(end_date)
print(start_date < end_date) # เปรียบเทียบวันเวลาได้
print(start_date > end_date) # เปรียบเทียบวันเวลาได้

2023-12-20 00:00:00
2023-12-25 00:00:00
True
False


ซึ่งรูปแบบของวันเวลา ดูได้ตาม[คู่มือ](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)ฟังก์ชัน `strftime()` ของภาษา `Python` ที่ `Pandas` เรียกใช้งาน

สำหรับรูปแบบของวันเวลาที่แสดงในตัวอย่าง เครื่องหมาย `%` แต่ละตัวมีความหมาย ดังตาราง

| คำสั่ง | ความหมาย | ตัวอย่าง |
|:--:|:--|:--|
| `%d` | วันที่ 2 หลัก  | `01, 02, …, 31` |
| `%m` | เลขเดือน 2 หลัก |  `01, 02, …, 12` |
| `%b` | ชื่อเดือนย่อ | `Jan, Feb, …, Dec` |
| `%B` | ชื่อเดือนเต็ม | `January, February, …, December` |
| `%y` | เลขปี 2 หลักท้าย (ละเลขศตวรรษ) | `00, 01, …, 99` |
| `%Y` | เลขปี 4 หลัก | `0001, 0002, …, 9999` |
| `%H`| ชั่วโมง | `00, 01, …, 23` |
| `%M`| นาที | `00, 01, …, 59` |
| `%S` |วินาที |`00, 01, …, 59` |



หากตัวแปรเป็นประเภท `datetime` แล้ว ก็จะสามารถเข้าถึงค่าวัน เดือน ปี ชั่วโมง นาที วินาที สัปดาห์ที่ ได้ ดังตัวอย่างนี้

In [ ]:
print(birth_time)
print('Year =', birth_time.year)
print('Month =', birth_time.month)
print('Day =', birth_time.day)
print('Week =', birth_time.week)
print('Hour =', birth_time.hour)
print('Minute =', birth_time.minute)
print('Second =', birth_time.second)

1998-03-19 15:34:56
Year = 1998
Month = 3
Day = 19
Week = 12
Hour = 15
Minute = 34
Second = 56


### การคำนวณส่วนต่างของเวลา

หากมีตัวแปร `datetime` 2 ตัว จะสามารถคำนวณความแตกต่างระหว่างวันเวลาได้ ดังเช่น

In [ ]:
print(f'start = {start_date}')
print(f'end   = {end_date}')
difference = end_date - start_date
print(difference)

start = 2023-12-20 00:00:00
end   = 2023-12-25 00:00:00
5 days 00:00:00


โดยผลลัพธ์ที่ได้ในตัวแปร `difference` จะได้ตัวแปรประเภท `timedelta`

In [ ]:
print(type(difference))

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>


ซึ่งส่วนต่าง 5 วัน ที่คำนวณได้ สามารถถึงค่าจำนวนวันออกมาได้ด้วย `.days`

In [ ]:
difference.days

5

### `datetime` ใน `Series/DataFrame`

`Series` หรือ `DataFrame` สามารถแปลงให้เป็น `datetime` ได้ โดยใช้ฟังก์ชัน `pd.to_datetime()` เช่นกัน แต่สำหรับข้อมูลร้านขายของออนไลน์ในคาบนี้ คอลัมน์ `InvoiceDate` ถูกแปลงเป็นประเภท `datetime` โดยอัตโนมัติอยู่แล้ว ดังนั้นจึงไม่ต้องทำอะไรเพิ่มเติม

ดังนั้นในส่วนนี้ จะยกตัวอย่างของการดึงค่าจากตัวแปร `datetime` ที่เป็นเลขสัปดาห์ที่, เลขเดือน, และเลขชั่วโมง ไปสร้างเป็น 3 คอลัมน์ใหม่

#### เพิ่มคอลัมน์เลขเดือน

In [ ]:
data['MonthNo'] = data['InvoiceDate'].dt.month

#### เพิ่มคอลัมน์เลขสัปดาห์

In [ ]:
data['WeekNo'] = data['InvoiceDate'].dt.isocalendar().week  # ใช้ Series.dt.week ไม่ได้

<ipython-input-306-bff456602fe4>:1: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



#### เพิ่มคอลัมน์สถานะว่าคำสั่งซื้อมาก่อน 12.00 น. หรือไม่


In [ ]:
data['BeforeNoon'] = (data['InvoiceDate'].dt.hour <= 12)

#### เพิ่มคอลัมน์สถานะว่าคำสั่งซื้อมาก่อนวันคริสต์มาสปี 2011 กี่วัน

In [ ]:
xmas = '2011-12-25'
data['DaysToXmas'] = (pd.to_datetime(xmas) - data['InvoiceDate']).dt.days  # เอาเฉพาะจำนวนวัน

#### ตอบคำถามว่า `InvoiceDate` แรกสุดและล่าสุดในคือวันและเวลาใด

In [ ]:
data['InvoiceDate'].agg(['min','max'])

min   2011-07-01 08:16:00
max   2011-08-31 17:45:00
Name: InvoiceDate, dtype: datetime64[ns]

## การรายงานข้อมูลเป็นรายลูกค้า

จากข้อมูลที่มี ในช่วง 2 เดือน จะตอบคำถามต่อไปนี้

1. ลูกค้าแต่ละคน มีจำนวน Invoice เกิดขึ้นกี่ Invoice และเกี่ยวข้องกับสินค้าทั้งหมดกี่ชนิด (`StockCode`)
2. ลูกค้าแต่ละคนเข้ามาแรกสุดและล่าสุดเมื่อใด (จากข้อมูลทั้งหมดที่มี)
3. ยอดซื้อรวมของลูกค้าแต่ละคน

ซึ่งทำได้ 2 วิธี





### **วิธีที่ 1** ทำ 1,2,และ 3 แล้วค่อยนำมารวมกัน

#### 1. ลูกค้าแต่ละคน มีจำนวน Invoice เกิดขึ้นกี่ Invoice และเกี่ยวข้องกับสินค้าทั้งหมดกี่ชนิด (`StockCode`)

In [ ]:
customer_invoiceno = data.groupby('CustomerID')[['InvoiceNo','StockCode']].nunique()
print(customer_invoiceno.shape)

(1540, 2)


#### 2. ลูกค้าแต่ละคนเข้ามาแรกสุดและล่าสุดเมื่อใด (จากข้อมูลทั้งหมดที่มี)

In [ ]:
customer_duration = data.groupby('CustomerID')['InvoiceDate'].agg(['min','max'])
print(customer_duration.shape)

(1540, 2)


#### 3. ยอดซื้อรวมของลูกค้าแต่ละคน

ก่อนอื่น ต้องคำนวณยอดรวมของแต่ละรายการก่อน ด้วยการนำราคาต่อหน่วยมาคูณกับจำนวนชิ้นที่ซื้อ

In [ ]:
data['Total'] = data['UnitPrice']*data['Quantity']
customer_total_spend = data.groupby('CustomerID')['Total'].agg(['sum'])

#### การนำคอลัมน์มาต่อกัน

จากต้นคาบ นักศึกษาได้ใช้ `pd.concat()` ไปแล้ว แต่เป็นการนำสองตารางมาต่อกันในแนวดิ่ง

สำหรับในหัวข้อนี้ เราต้องการนำตารางมาต่อกันในแนวราบ กล่าวคือ แต่ละแถวที่มี `CustomerID` ตรงกัน จะถูกนำมาต่อกัน ซึ่งสามารถทำได้ด้วยคำสั่ง `pd.concat(..., axis=1)` ที่เพิ่มพารามิเตอร์ `axis=1` เข้ามา

อนึ่ง ใน `Pandas` นั้น มีการนิยามแกน (axis) เป็นค่า ดังนี้
- `axis=0` หมายถึงแกนนอน (กระทำกับคอลัมน์ที่ชื่อเหมือนกัน) (มักใช้เป็นค่า **default** ในหลายฟังก์ชัน)
- `axis=1` หมายถึงแกนตั้ง (กระทำกับแถวที่ index เหมือนกัน) (โดยปกติมักจะต้องระบุเอง)



In [ ]:
pd.concat( (customer_invoiceno, customer_total_spend, customer_duration), axis=1)

,InvoiceNo,StockCode,sum,min,max
CustomerID,,,,,
12347,1,22,584.91,2011-08-02 08:48:00,2011-08-02 08:48:00
12358,1,12,484.86,2011-07-12 10:04:00,2011-07-12 10:04:00
12360,1,30,534.70,2011-08-19 10:10:00,2011-08-19 10:10:00
12362,3,46,755.11,2011-07-07 12:32:00,2011-08-19 10:38:00
12363,1,7,252.90,2011-08-22 10:18:00,2011-08-22 10:18:00
...,...,...,...,...,...
18265,1,29,488.55,2011-07-04 15:26:00,2011-07-04 15:26:00
18268,2,1,0.00,2011-07-28 11:16:00,2011-07-28 19:13:00
18272,2,50,1125.91,2011-07-12 15:42:00,2011-08-18 12:06:00


### **วิธีที่ 2** การทำ Aggregration โดยการระบุฟังก์ชันจำเพาะกับแต่ละคอลัมน์

วิธีการนี้ ทำได้โดยการส่ง Dictionary ให้กับ `.agg` ดังนี้

In [ ]:
data.groupby('CustomerID').agg(
    {'InvoiceNo':['nunique'],
     'StockCode':['nunique'],
     'Total':['sum'],
     'InvoiceDate':['min','max'],
     }
)

InvoiceNo StockCode    Total         InvoiceDate  \
             nunique   nunique      sum                 min   
CustomerID                                                    
12347              1        22   584.91 2011-08-02 08:48:00   
12358              1        12   484.86 2011-07-12 10:04:00   
12360              1        30   534.70 2011-08-19 10:10:00   
12362              3        46   755.11 2011-07-07 12:32:00   
12363              1         7   252.90 2011-08-22 10:18:00   
...              ...       ...      ...                 ...   
18265              1        29   488.55 2011-07-04 15:26:00   
18268              2         1     0.00 2011-07-28 11:16:00   
18272              2        50  1125.91 2011-07-12 15:42:00   
18282              2         7    98.76 2011-08-05 13:35:00   
18283              2        54   143.19 2011-07-14 13:00:00   

                                
                           max  
CustomerID                      
12347      2011-08-02 08:48:00  
12358      2011-07-12 10:04:00  
12360      2011-08-19 10:10:00  
12362      2011-08-19 10:38:00  
12363      2011-08-22 10:18:00  
...                        ...  
18265      2011-07-04 15:26:00  
18268      2011-07-28 19:13:00  
18272      2011-08-18 12:06:00  
18282      2011-08-09 15:10:00  
18283      2011-07-14 13:20:00  

[1540 rows x 5 columns]

**ลองทำ**

ก่อนจบจากส่วนนี้ และเข้าสู่เนื้อหาของการวาดกราฟ ขอให้นักศึกษาศึกษา การใช้งานฟังก์ชัน `to_excel()` ตามลิงค์นี้

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html

เพื่อบันทึกตารางผลลัพธ์ที่ได้จากการทำ aggregation ลงไฟล์ Excel จากนั้นลองดาวน์โหลดไฟล์ที่ได้ ไปเปิดในโปรแกรม Excel ในเครื่องคอมพิวเตอร์ของนักศึกษา

## การวาดกราฟ

ในหัวข้อนี้ จะมุ่งเน้นที่การวาดกราฟแบบต่าง ๆ  แต่เนื่องจากข้อมูลที่มีจำเป็นต้องทำการ clean ก่อน ซึ่งจะทำดังนี้

### Data cleansing

การทำความสะอาดข้อมูล จะทำความสะอาดอะไรบ้าง จำเป็นจะต้องทราบก่อนว่าจะวิเคราะห์ไปเพื่ออะไร สมมติว่าสนใจวิเคราะห์เกี่ยวกับพฤติกรรมของลูกค้า ดังนั้น จึงขอทำความสะอาดข้อมูลดังนี้

1. เนื่องจากสนใจเฉพาะแถวที่เป็นคำสั่งซื้อของลูกค้า ดังนั้นจึงจำเป็นต้องมี `CustomerID` จึงลบแถวที่มี `CustomerID` เป็น `NaN` ออกไป
2. ลบ `InvoiceNo` ที่ขึ้นต้นด้วย `C` และ `A` เพราะคำสั่งซื้อที่ถูกยกเลิกถูกลง `UnitPrice` เป็น `0`
3. ไม่สนใจ `UnitPrice` เท่าับ `0` เพราะอาจเป็นของแจกฟรี (สมมติฐาน)

In [ ]:
df = data.dropna(subset=['CustomerID'])  # ข้อ 1
df = df[~(df['InvoiceNo'].str.startswith('A') | df['InvoiceNo'].str.startswith('C'))]  # ข้อ 2
df = df[df['UnitPrice'] > 0]  # ข้อ 3

### ลูกค้ามาจากไหนบ้าง

ก่อนอื่น นักศึกษาลองหาสิว่า จากข้อมูลในตัวแปร `df`
- มีลูกค้าทั้งหมดกี่ราย
- มีประเทศที่เป็นไปได้ทั้งหมด กี่ประเทศ

#### การแจกแจงจำนวนลูกค้าตามรายประเทศ

In [ ]:
import plotly.express as px
per_customer = df.drop_duplicates(subset=['Country','CustomerID'])
n_customers = per_customer.groupby('Country')['CustomerID'].count()  # ได้ Series
px.bar(n_customers, text_auto=True, title='การแจกแจงจำนวนลูกค้าตามรายประเทศ แบบที่ 1')

แต่เนื่องจาก United Kingdom มีมากเกินไป ทำให้อ่านกราฟได้ยาก จึงขอตัดออกจากกราฟเลย ด้วยคำสั่ง `Series.drop()` ดังนี้

In [ ]:
n_customers = per_customer.groupby('Country')['CustomerID'].count().drop('United Kingdom')
px.bar(n_customers, text_auto=True, title='การแจกแจงจำนวนลูกค้าตามรายประเทศ แบบที่ 2 (exclude UK)')

หรืออีกทางเลือกหนึ่งคือ ใช้แกนตั้งให้เป็น log scale โดยใช้พารามิเตอร์ `log_y=True`

In [ ]:
n_customers = per_customer.groupby('Country')['CustomerID'].count()
px.bar(n_customers, text_auto=True, log_y=True, title='การแจกแจงจำนวนลูกค้าตามรายประเทศ แบบที่ 3 (log-scale)')

หรือจะลองใช้ Pie chart ([คู่มือ](https://plotly.com/python/pie-charts/)) แทน Bar chart

รูปแบบการใช้คำสั่งสร้าง Pie chart คือ

```python
px.pie(ข้อมูลที่เป็นDataFrame, values=ชื่อคอลัมน์ที่เก็บตัวเลข, names=ชื่อกำกับตัวเลข)
```



In [ ]:
n_customers = ( per_customer.groupby('Country')['CustomerID']
                    .nunique()      # ได้ Series ที่มี Country เป็น index โดยที่ Series ถูกตั้งชื่อว่า CustomerID
                    .rename('No of customers')  # ไม่ต้องการให้ Series ชื่อ CustomerID จึงแก้ชื่อเป็น No of customers
                    .drop('United Kingdom')  # เอาค่าของ index='United Kingdom' ออกจาก Series
                    .reset_index()    # ทำให้ Series กลายเป็น DataFrame  ค่าของ Series จะกลายมาเป็นคอลัมน์ชื่อ No of customers
)
n_customers.head()

,Country,No of customers
0,Australia,3
1,Austria,4
2,Belgium,10
3,Canada,2
4,Channel Islands,4


In [ ]:
px.pie(n_customers, values='No of customers', names='Country',  title='การแจกแจงสัดส่วนลูกค้าตามรายประเทศ แบบที่ 4 (exclude UK)')

### คำสั่งซื้อจากแต่ละประเทศ เกิดขึ้นก่อนเที่ยงหรือหลังเที่ยง

ตัวอย่างนี้เป็นเพียงตัวอย่างการวาดกราฟเท่านั้น แต่ข้อเท็จจริงที่นำไปวาดกราฟอาจไม่ถูกต้อง เนื่องจากไม่ทราบว่า `InvoiceDate` คือเวลาใน Time zone ไหน จึงขอตั้งสมมติฐานว่าเป็นเวลาของ UK เนื่องจากบริษัทจดทะเบียนที่ UK และลูกค้าส่วนใหญ่มาจาก UK

In [ ]:
df2 = df.drop_duplicates(subset=['InvoiceNo','Country','BeforeNoon'])
df2 = df2[['InvoiceNo','Country','BeforeNoon']].sort_values(by='Country')

In [ ]:
fig = px.histogram(df2, x='Country',
                   color='BeforeNoon',
                   barmode='group',
                   text_auto=True,
                   log_y=True)
fig.show()

หากสงสัยว่ากราฟที่วาด ผิดหรือไม่ ก็ตรวจสอบได้จากคำสั่งของ `Pandas` อีกทาง

In [ ]:
df2.groupby('Country')['BeforeNoon'].value_counts()

Country             BeforeNoon
Australia           True             5
                    False            4
Austria             True             3
                    False            1
Belgium             False            8
                    True             6
Canada              True             3
Channel Islands     True             4
                    False            1
Denmark             True             2
EIRE                False           21
                    True            18
European Community  True             1
Finland             True             6
France              True            35
                    False           15
Germany             True            39
                    False           33
Greece              False            1
Iceland             True             1
Israel              True             2
Italy               False            3
                    True             2
Japan               False            1
                    True         

### ปริมาณคำสั่งซื้อต่อวันเป็นอย่างไรบ้าง

ขอให้นักศึกษาลองคิดดูว่า การจะคำนวณปริมาณคำสั่งซื้อในแต่ละวัน จะทำได้อย่างไร นักศึกษาอาจดึงเฉพาะวันที่ออกจาก `InvoiceDate` (เพราะไม่อยากได้ค่าเวลา) ไปสร้างเป็นคอลัมน์ใหม่แล้ว ซึ่งหากนำวันที่มาจัดกลุ่มด้วย `groupby` ก็จะได้กลุ่มย่อยของ `InvoiceNo` เป็นรายวัน ซึ่งสามารถนับจำนวน `InvoiceNo` ที่ไม่ซ้ำกันในหนึ่งวัน ได้ด้วย `nunique()` ดังนี้



In [ ]:
df['InvoiceDateOnly'] = df['InvoiceDate'].dt.date   # dt.date จะใด้ datetime แต่ dt.day จะได้เลขวันที่
invoice_by_day = df.groupby('InvoiceDateOnly')['InvoiceNo'].nunique()
px.line(invoice_by_day, markers=True, title='ปริมาณคำสั่งซื้อในแต่ละวัน (แบบที่ 1 ใช้ groupby)')

อย่างไรก็ตาม กราฟที่ได้จะข้ามบางวันไป ซึ่งดูจากกราฟอาจจะดูไม่ค่อยออก ดังนั้น ในหลายกรณี จึงจำเป็นที่จะต้องเติมวันที่ที่ขาดหายไปเข้าไปด้วย ดังนั้นจึงเป็นที่มาของการวาดกราฟแบบที่ 2 ที่ใช้คำสั่ง `set_index()` ตามด้วย `resample()`

#### คำสั่ง `set_index()`

เป็นคำสั่งที่ใช้กำหนดให้คอลัมน์ที่ต้องการ กลายเป็น `index` ของ `DataFrame`

In [ ]:
df2 = df.set_index('InvoiceDate')
df2.head()

,index,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Sheet,MonthNo,WeekNo,BeforeNoon,DaysToXmas,Total,InvoiceDateOnly
InvoiceDate,,,,,,,,,,,,,,,
2011-07-01 08:16:00,0,558638,84836,ZINC METAL HEART DECORATION,12,1.25,16317,United Kingdom,July,7,26,True,176,15.00,2011-07-01
2011-07-01 08:16:00,1,558638,71459,HANGING JAM JAR T-LIGHT HOLDER,24,0.85,16317,United Kingdom,July,7,26,True,176,20.40,2011-07-01
2011-07-01 08:16:00,2,558638,22784,LANTERN CREAM GAZEBO,3,4.95,16317,United Kingdom,July,7,26,True,176,14.85,2011-07-01
2011-07-01 08:16:00,3,558638,23145,ZINC T-LIGHT HOLDER STAR LARGE,12,0.95,16317,United Kingdom,July,7,26,True,176,11.40,2011-07-01
2011-07-01 08:16:00,4,558638,22674,FRENCH TOILET SIGN BLUE METAL,12,1.25,16317,United Kingdom,July,7,26,True,176,15.00,2011-07-01


จะเห็นว่า ตอนนี้ `InvoiceDate` ถูกเปลี่ยนให้เป็น index โดยที่คอลัมน์ `InvoiceDate` เดิม และ index เดิมก็หายไปด้วย ทั้งนี้มีข้อสังเกตุคือ การที่ `InvoiceDate` มีค่าซ้ำกันได้  ทำให้ index ที่ได้มีค่าไม่เป็นเอกลักษณ์เฉพาะแถว

เมื่อได้ index ที่เป็นวันเวลา โดยที่วันเวลาถูกเรียงลำดับจากน้อยไปมากไว้ก่อนแล้ว (หากยังไม่เรียง ก็ให้ใช้ `DataFrame.sort_index()` ก่อน) จะสามารถใช้คำสั่ง `resample() ได้ ` ([คู่มือ](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling)  ซึ่ง `resample()` จะทำงานคล้าย `groupby` และยังสามารถเติมวันเวลาที่ขาดหายเข้าไปด้วย  

สำหรับในตัวอย่างต่อไปนี้ `.resample('D')` จะเป็นการไล่วันเวลาในระดับวันที่

`2011-07-01`, `2011-07-02`, `2011-07-03`, ...

ซึ่งสามารถเป็นการไล่เวลาในระดับชั่วโมง, สัปดาห์, เดือน เป็นต้น ซึ่งสามารถดูเพิ่มเติมได้จากคู่มือ

หลังจาก `resample('D')`  แล้ว จึงทำการเลือกเฉพาะคอลัมน์ `InvoiceNo` และ aggregrate ข้อมูลในแต่ละวัน ด้วยคำสั่ง `nunique()` เพื่อนับจำนวน `InvoiceNo` ที่ไม่ซ้ำกันในวันใด ๆ

In [ ]:
df3 = df2.resample("D")['InvoiceNo'].nunique()
df3

InvoiceDate
2011-07-01    42
2011-07-02     0
2011-07-03    25
2011-07-04    37
2011-07-05    64
              ..
2011-08-27     0
2011-08-28    37
2011-08-29     0
2011-08-30    23
2011-08-31    42
Freq: D, Name: InvoiceNo, Length: 62, dtype: int64

จากคำอธิบายข้างต้น สามารถนำมาสรุปเขียนเป็นคำสั่งในคราวเดียว และวาดกราฟเส้น ได้ ดังนี้

In [ ]:
invoice_by_day = (df.set_index('InvoiceDate')
                      .resample("D")["InvoiceNo"]
                      .nunique()
)
px.line(invoice_by_day, markers=True, title='ปริมาณคำสั่งซื้อในแต่ละวัน (แบบที่ 2 ใช้ resample)')

จากกราฟ จะเห็นวันที่จำนวน Invoice เป็น 0 มันคือวันอะไร?

**จบเนื้อหา**

นักศึกษาลองสรุปซิว่า วันนี้นักศึกษาได้เรียนรู้คำสั่งอะไรบ้าง